In [1]:
import tools
corpus = tools.load_corpus()
from importlib import reload as rl
import non_iid_dataloader as niiddl, lhotse
partition = niiddl.prepare_partition(corpus['train'])
from tqdm import tqdm
from torch.nn import TransformerDecoder, TransformerDecoderLayer, Transformer
import torch
import x_transformers, torch
tk = tools.load_tokenizer()
from omegaconf.omegaconf import OmegaConf
%cd tedlium
import lm_utils
tokenizer = tk

[NeMo W 2022-10-27 18:39:15 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-10-27 18:39:17 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-10-27 18:39:17 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-10-27 18:39:17 nemo_logging:349] /exp/exp1/acp21rjf/lhotse/lhotse/lazy.py:388: UserWarning: A lambda was passed to LazyMapper: it may prevent you from forking this process. If you experience issues with num_workers > 0 in torch.utils.data.DataLoader, try passing a regular function instead.
      warnings.warn(
    


[Errno 2] No such file or directory: 'tedlium'
/exp/exp1/acp21rjf/deliberation/speachy/tedlium


In [2]:
meetings = niiddl.prepare_partition(corpus['train'])

In [12]:
import lhotse
import non_iid_dataloader as niiddl
rl(niiddl),rl(lhotse)
from lhotse.dataset.collation import collate_audio
from lhotse.dataset.cut_transforms import plain_concat, individual_speaker_concat
niiddl.plain_concat = plain_concat
niiddl.individual_speaker_concat = individual_speaker_concat

samples = niiddl.prepare_samples(meetings, max_allowed_utterance_gap=5.0, max_duration=100)


In [2]:
rl(tools)
print(torch.cuda.is_available())

True


In [3]:
import model_utils
class argsclass:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [4]:
rl(lm_utils)
model = lm_utils.load_model(lm_utils.load_config('./lm/decoder_test.yaml'), tools.load_tokenizer(), max_len=1862)

In [5]:
model.to('cpu')

TransformerWrapper(
  (token_emb): TokenEmbedding(
    (emb): Embedding(128, 256)
  )
  (post_emb_norm): Identity()
  (emb_dropout): Dropout(p=0.0, inplace=False)
  (project_emb): Identity()
  (attn_layers): Decoder(
    (layers): ModuleList(
      (0): ModuleList(
        (0): ModuleList(
          (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (1): None
          (2): None
        )
        (1): Attention(
          (to_q): Linear(in_features=256, out_features=512, bias=False)
          (to_k): Linear(in_features=256, out_features=512, bias=False)
          (to_v): Linear(in_features=256, out_features=512, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
          (to_out): Linear(in_features=512, out_features=256, bias=False)
        )
        (2): Residual()
      )
      (1): ModuleList(
        (0): ModuleList(
          (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (1): None
          (2): None
        )
        (1): Fee

In [6]:
!ls checkpoints_LM3

checkpoint_46_id_32.pt	checkpoint_50_id_80.pt	checkpoint_52_id_26.pt
checkpoint_47_id_60.pt	checkpoint_51_id_44.pt


In [7]:
epoch, val_loss  = model_utils.load_checkpoint(args=argsclass(**{'checkpoint': './checkpoints_15_mediumplus_tlm3000vocab/checkpoint_7_id_13.pt'}), model=model, force_cpu=True)

./checkpoints_15_mediumplus_tlm3000vocab/checkpoint_7_id_13.pt
odict_keys(['token_emb.emb.weight', 'attn_layers.layers.0.0.0.weight', 'attn_layers.layers.0.0.0.bias', 'attn_layers.layers.0.1.to_q.weight', 'attn_layers.layers.0.1.to_k.weight', 'attn_layers.layers.0.1.to_v.weight', 'attn_layers.layers.0.1.to_out.weight', 'attn_layers.layers.1.0.0.weight', 'attn_layers.layers.1.0.0.bias', 'attn_layers.layers.1.1.net.0.0.weight', 'attn_layers.layers.1.1.net.0.0.bias', 'attn_layers.layers.1.1.net.3.weight', 'attn_layers.layers.1.1.net.3.bias', 'attn_layers.layers.2.0.0.weight', 'attn_layers.layers.2.0.0.bias', 'attn_layers.layers.2.1.to_q.weight', 'attn_layers.layers.2.1.to_k.weight', 'attn_layers.layers.2.1.to_v.weight', 'attn_layers.layers.2.1.to_out.weight', 'attn_layers.layers.3.0.0.weight', 'attn_layers.layers.3.0.0.bias', 'attn_layers.layers.3.1.net.0.0.weight', 'attn_layers.layers.3.1.net.0.0.bias', 'attn_layers.layers.3.1.net.3.weight', 'attn_layers.layers.3.1.net.3.bias', 'attn_lay

RuntimeError: Error(s) in loading state_dict for TransformerWrapper:
	size mismatch for token_emb.emb.weight: copying a param with shape torch.Size([3000, 256]) from checkpoint, the shape in current model is torch.Size([128, 256]).
	size mismatch for to_logits.weight: copying a param with shape torch.Size([3000, 256]) from checkpoint, the shape in current model is torch.Size([128, 256]).
	size mismatch for to_logits.bias: copying a param with shape torch.Size([3000]) from checkpoint, the shape in current model is torch.Size([128]).

In [8]:
torch.cuda.is_available()

True

In [14]:
model.greedy_generate('A prominent limitation of ', tokenizer=tokenizer, temperature=1.0)

'a prominent limitation of the largest clons o'

In [9]:
import non_iid_dataloader as niiddl
import lm_utils
rl(niiddl)
rl(lm_utils)

<module 'lm_utils' from '/exp/exp1/acp21rjf/deliberation/speachy/tedlium/lm_utils.py'>

In [ ]:
durations = [
    0.0,
    15.0,
    30.0,
    45.0,
    60.0,
    75.0,
    90.0,
    100.0,
    120.0,
    140.0,
    180.0,
    200.0,
    250.0,
    300.0,
    400.0,
    500.0,
]
for duration in durations:
    dl = niiddl.get_data_loader(
        corpus['test'], 
        tokenizer=tokenizer, 
        batch_size=100, 
        shuffle=False,
        max_duration=duration,
        text_only=True,
    )
    ppl, avg_len = lm_utils.eval_corpus_perplexity(model, dl, device='cpu')
    print(f"Duration: {duration}, PPL: {ppl}, Avg Len: {avg_len}")

In [8]:
dl = niiddl.get_data_loader(
    corpus['train'], 
    tokenizer=tokenizer, 
    batch_size=20, 
    shuffle=True,
    max_duration=100,
    text_only=True,
)

In [9]:
for batch in dl:
    z = batch
    break

In [ ]:
x = lm_utils.eval_corpus_perplexity(model, dl, device='cpu')
x

In [436]:
#ppl, avg_len = lm_utils.eval_corpus_perplexity(model, dl, device='cpu')
#print(ppl, avg_len)

perplexity: 417.11: 100%|██████████| 26/26 [01:22<00:00,  3.17s/it] 

479.1307067871094 1441.75


In [339]:
lm_utils.eval_corpus_perplexity(model, dl, device='cpu')

perplexity: 459.58: 100%|██████████| 124/124 [00:18<00:00,  6.71it/s] 


167.10165405273438

In [158]:
lm_utils.greedy_generate(model, tokenizer, 'how are', 10, force_cpu=True, temperature=0)

[0, 17, 54, 97, 3, 107, 3, 107, 3, 107]


'⁇  how are as as as'

In [ ]:
model

In [159]:
rl(lm_utils)

<module 'lm_utils' from '/exp/exp1/acp21rjf/deliberation/speachy/tedlium/lm_utils.py'>

In [68]:
tokenizer.text_to_ids('hello there')

[78, 30, 104, 6, 8]

In [75]:
tokenizer.add_special_tokens(['<sos>', '<eos>'])

In [ ]:
tokenizer.additional_special_tokens_ids()

In [74]:
tokenizer.ids_to_text(out[0)

'en'

In [5]:
tokens = torch.randn(2,3)
token_lens = torch.tensor([2, 3])

NameError: name 'torch' is not defined

In [177]:
def add_eos(tkn, tkn_len, eos_id = 30):
    tkn = tkn.clone() 
    tkn[torch.arange(tkn.shape[0], device=tkn_len.device, dtype=torch.int32), (tkn_len-1.0).to(torch.int32)] = eos_id
    return tkn

token_lens.to(torch.int64)

tensor([2, 3])

In [181]:
[0] + tokenizer.text_to_ids('hello')

[0, 78, 30, 104]

In [178]:
add_eos(tokens, token_lens)

IndexError: tensors used as indices must be long, byte or bool tensors

In [145]:
bt = torch.tensor(
    [[0,0,0],
    [0,0,0]]
, dtype=torch.bool)

In [146]:
tokens[~bt]

tensor([ 0.3837,  0.5211, -1.2230,  0.5048, -0.1723, -2.0018])

In [39]:
from lm.s4 import S4

In [43]:
model  =S4(d_model=256)

In [54]:
emb = torch.nn.Embedding(100, 256)

In [58]:
emb(torch.randint(0, 100, (2,3))).shape

torch.Size([2, 3, 256])

In [61]:
from einops import rearrange



In [143]:
rl(lm_utils)
from lm_utils import S4adapter

In [301]:
get_num_params(
    S4adapter(S4(d_model=712, measure='legs', mode='nplr', transposed=False, d_state=64), vocab_size=128)
) / 1e6 

1.381408

In [139]:
model = S4adapter(S4(d_model=2048, measure='legs', mode='nplr', transposed=False, d_state=512), vocab_size=128)

In [140]:
model

S4adapter(
  (model): S4(
    (kernel): SSKernel(
      (kernel): SSKernelNPLR()
    )
    (activation): GELU()
    (dropout): Identity()
    (output_linear): Sequential(
      (0): Linear(in_features=2048, out_features=4096, bias=True)
      (1): GLU(dim=-1)
    )
  )
  (embedding): Embedding(128, 2048)
  (predict): Linear(in_features=2048, out_features=128, bias=True)
  (softmax): Softmax(dim=-1)
)

In [134]:

model(torch.randint(0, 100, (2,3))).shape

torch.Size([2, 3, 2048]) emb
torch.Size([2, 3, 2048]) s4
torch.Size([2, 3, 128]) logits


torch.Size([2, 3, 128])

In [129]:
torch.randint(0, 100, (2,3)).shape

torch.Size([2, 3])